# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fc674495f70>
├── 'a' --> tensor([[ 1.0231,  1.0387, -0.3989],
│                   [ 1.1693,  0.3416, -0.2755]])
└── 'x' --> <FastTreeValue 0x7fc6744346a0>
    └── 'c' --> tensor([[ 0.2037, -1.3397,  1.7660, -1.3289],
                        [ 0.5559, -0.8278,  1.0283, -0.5832],
                        [ 2.5793, -1.1035,  0.5420, -0.5164]])

In [4]:
t.a

tensor([[ 1.0231,  1.0387, -0.3989],
        [ 1.1693,  0.3416, -0.2755]])

In [5]:
%timeit t.a

61.5 ns ± 0.0345 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fc674495f70>
├── 'a' --> tensor([[ 0.9322, -0.3561, -1.9322],
│                   [ 0.0286, -0.6647,  0.8688]])
└── 'x' --> <FastTreeValue 0x7fc6744346a0>
    └── 'c' --> tensor([[ 0.2037, -1.3397,  1.7660, -1.3289],
                        [ 0.5559, -0.8278,  1.0283, -0.5832],
                        [ 2.5793, -1.1035,  0.5420, -0.5164]])

In [7]:
%timeit t.a = new_value

58.5 ns ± 0.0521 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.0231,  1.0387, -0.3989],
               [ 1.1693,  0.3416, -0.2755]]),
    x: Batch(
           c: tensor([[ 0.2037, -1.3397,  1.7660, -1.3289],
                      [ 0.5559, -0.8278,  1.0283, -0.5832],
                      [ 2.5793, -1.1035,  0.5420, -0.5164]]),
       ),
)

In [10]:
b.a

tensor([[ 1.0231,  1.0387, -0.3989],
        [ 1.1693,  0.3416, -0.2755]])

In [11]:
%timeit b.a

56.6 ns ± 0.0521 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.2762,  0.5428, -1.4572],
               [-1.5385, -0.0950, -1.2951]]),
    x: Batch(
           c: tensor([[ 0.2037, -1.3397,  1.7660, -1.3289],
                      [ 0.5559, -0.8278,  1.0283, -0.5832],
                      [ 2.5793, -1.1035,  0.5420, -0.5164]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.0911 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

950 ns ± 0.0816 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 26.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 535 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 388 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fc6744275b0>
├── 'a' --> tensor([[[ 1.0231,  1.0387, -0.3989],
│                    [ 1.1693,  0.3416, -0.2755]],
│           
│                   [[ 1.0231,  1.0387, -0.3989],
│                    [ 1.1693,  0.3416, -0.2755]],
│           
│                   [[ 1.0231,  1.0387, -0.3989],
│                    [ 1.1693,  0.3416, -0.2755]],
│           
│                   [[ 1.0231,  1.0387, -0.3989],
│                    [ 1.1693,  0.3416, -0.2755]],
│           
│                   [[ 1.0231,  1.0387, -0.3989],
│                    [ 1.1693,  0.3416, -0.2755]],
│           
│                   [[ 1.0231,  1.0387, -0.3989],
│                    [ 1.1693,  0.3416, -0.2755]],
│           
│                   [[ 1.0231,  1.0387, -0.3989],
│                    [ 1.1693,  0.3416, -0.2755]],
│           
│                   [[ 1.0231,  1.0387, -0.3989],
│                    [ 1.1693,  0.3416, -0.2755]]])
└── 'x' --> <FastTreeValue 0x7fc674434130>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.8 µs ± 37.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fc6488eb310>
├── 'a' --> tensor([[ 1.0231,  1.0387, -0.3989],
│                   [ 1.1693,  0.3416, -0.2755],
│                   [ 1.0231,  1.0387, -0.3989],
│                   [ 1.1693,  0.3416, -0.2755],
│                   [ 1.0231,  1.0387, -0.3989],
│                   [ 1.1693,  0.3416, -0.2755],
│                   [ 1.0231,  1.0387, -0.3989],
│                   [ 1.1693,  0.3416, -0.2755],
│                   [ 1.0231,  1.0387, -0.3989],
│                   [ 1.1693,  0.3416, -0.2755],
│                   [ 1.0231,  1.0387, -0.3989],
│                   [ 1.1693,  0.3416, -0.2755],
│                   [ 1.0231,  1.0387, -0.3989],
│                   [ 1.1693,  0.3416, -0.2755],
│                   [ 1.0231,  1.0387, -0.3989],
│                   [ 1.1693,  0.3416, -0.2755]])
└── 'x' --> <FastTreeValue 0x7fc6488eb580>
    └── 'c' --> tensor([[ 0.2037, -1.3397,  1.7660, -1.3289],
                        [ 0.5559, -0.8278,  1.0283, -0.5832],
                 

In [23]:
%timeit t_cat(trees)

29.5 µs ± 88.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.5 µs ± 127 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.2037, -1.3397,  1.7660, -1.3289],
                       [ 0.5559, -0.8278,  1.0283, -0.5832],
                       [ 2.5793, -1.1035,  0.5420, -0.5164]],
              
                      [[ 0.2037, -1.3397,  1.7660, -1.3289],
                       [ 0.5559, -0.8278,  1.0283, -0.5832],
                       [ 2.5793, -1.1035,  0.5420, -0.5164]],
              
                      [[ 0.2037, -1.3397,  1.7660, -1.3289],
                       [ 0.5559, -0.8278,  1.0283, -0.5832],
                       [ 2.5793, -1.1035,  0.5420, -0.5164]],
              
                      [[ 0.2037, -1.3397,  1.7660, -1.3289],
                       [ 0.5559, -0.8278,  1.0283, -0.5832],
                       [ 2.5793, -1.1035,  0.5420, -0.5164]],
              
                      [[ 0.2037, -1.3397,  1.7660, -1.3289],
                       [ 0.5559, -0.8278,  1.0283, -0.5832],
                       [ 2.5793, -1.1035,  0.5420, -0.5164]],

In [26]:
%timeit Batch.stack(batches)

78.5 µs ± 83.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.2037, -1.3397,  1.7660, -1.3289],
                      [ 0.5559, -0.8278,  1.0283, -0.5832],
                      [ 2.5793, -1.1035,  0.5420, -0.5164],
                      [ 0.2037, -1.3397,  1.7660, -1.3289],
                      [ 0.5559, -0.8278,  1.0283, -0.5832],
                      [ 2.5793, -1.1035,  0.5420, -0.5164],
                      [ 0.2037, -1.3397,  1.7660, -1.3289],
                      [ 0.5559, -0.8278,  1.0283, -0.5832],
                      [ 2.5793, -1.1035,  0.5420, -0.5164],
                      [ 0.2037, -1.3397,  1.7660, -1.3289],
                      [ 0.5559, -0.8278,  1.0283, -0.5832],
                      [ 2.5793, -1.1035,  0.5420, -0.5164],
                      [ 0.2037, -1.3397,  1.7660, -1.3289],
                      [ 0.5559, -0.8278,  1.0283, -0.5832],
                      [ 2.5793, -1.1035,  0.5420, -0.5164],
                      [ 0.2037, -1.3397,  1.7660, -1.3289],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 369 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

304 µs ± 1.06 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
